<a href="https://colab.research.google.com/github/AtlasLearner/manga_stream/blob/main/manga_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import re
import urllib.request
from bs4 import BeautifulSoup
import requests
from google.colab import files

In [2]:
#!/usr/bin/python
import os, sys

# Path to be created
hpath = '/content/'

In [3]:
# Logging to see requests status
import logging

import http.client
 # Set to 1 for debug mode
http.client.HTTPConnection.debuglevel = 0

# You must initialize logging, otherwise you'll not see debug output.
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = False

requests.get("https://www.google.com")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.google.com:443
DEBUG:urllib3.connectionpool:https://www.google.com:443 "GET / HTTP/1.1" 200 None


<Response [200]>

# Variables

In [21]:
# Store chapter and respective links
chapters = []

# Store chapter names that have been downloaded
completed = []


# One Piece Chapter Link
OP_URL  = "https://onepiecechapters.com/manga/one-piece-chapter-{}/"
BNHA_URL = "https://w20.readheroacademia.com/manga/boku-no-hero-academia-chapter-{}/."

# File Naming System
OP_CN = "One_Piece_-_Chapter_"
BNHA_CN = "Boku_no_Hero_Academia_-_Chapter_"

## Get all img_links using Beautiful Soup

In [4]:
def get_img_links(start, end, url, CN):
    print("Running\n")
    for j in range(start, end + 1):
        print("Current Chapter: ", j)
        URL = url.format(j)
        print(URL + "\n")
        request = requests.get(URL)
        if request.status_code == 200:
            # requests.get()
            #   - If slow
            #       1) Use header - Simulates as a website (the intended use) for websites that prevent web scraping
            #       2) Use timeout = 5 due to ipv4 vs ipv6
            page = requests.get(URL)
            
            page_content = BeautifulSoup(page.content, 'html.parser')
            row_data = [j, CN]
            
            # Finds all src/links of pictures in <img> tags
            for row in page_content.findAll('img'):
                link = row.attrs.get("src")
                link = re.findall('[^.]ttps://.*jpg|[^.]ttps://.*png', link)
                row_data.append(link)

            # Removes invalid links based on regex expression
            for i,x in enumerate(row_data):
                if x==[]:
                    row_data.remove([])

            chapters.append(row_data)

            
        else:
            print("Fail to connect. #FUCKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK")
            break

    print(chapters)

## Download all chapters with their respective img links

In [63]:
def dl_chapters():
    for j in range(len(chapters)):
        print(j)
        for i in range(len(chapters[j])):
            if i == 0:
                chapter_name = str(chapters[j][i+1]) + str(chapters[j][i]) + '_-_'
                completed.append(chapter_name)
                os.mkdir(chapter_name)
                os.chdir(chapter_name)
                !pwd
                continue
            elif i == 1:
                continue
            else:
                Image_URL=''.join(map(str, chapters[j][i]))

                filename = re.search(r'/([\+\%\w_-]+[.](jpg|gif|png))$', Image_URL)
                if not filename:
                    print("Regex didn't match with the url: {}".format(Image_URL))
                    continue
                
                with open('{0:02d}'.format(i-1) + " - " + filename.group(1), 'wb') as f:
                    response = requests.get(Image_URL)
                    if response.status_code == 200:
                        f.write(response.content)
                        f.close()
                    
                    else:
                    # .group() gives the group based on an index on the regex expression - so 0 is the whole thing, 1 is the group onwards, 2 is (jpg|gif|png)
                        print(filename.group(1) + " - FAILED")

        os.chdir(hpath)
        os.system('zip -r /content/' + chapter_name + '.zip ' + chapter_name)
        files.download(chapter_name + '.zip')


    print("Downloaded chapters: ")
    print(completed)


## Run the program
Remember to change chapter number every week - last updated 22/11/2020


In [6]:
# Run this where parameters are number of start of chapter, end of chapter, URL, naming

get_img_links(995, 996, OP_URL, OP_CN)
get_img_links(260, 280, BNHA_URL, BNHA_CN)

dl_chapters()

Running

Current Chapter:  995
https://onepiecechapters.com/manga/one-piece-chapter-995/

<Response [200]>
Current Chapter:  996
https://onepiecechapters.com/manga/one-piece-chapter-996/

<Response [200]>
[[995, 'One_Piece_-_Chapter_', ['https://onepiecechapters.com/wp-content/uploads/2020/05/OPthub.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/995+Page+0.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/995+Page+1.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/995+Page+2.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/op995%20Page%203v2.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/op_995%20Page%204-5.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/op_995%20Page%206.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/995+Page+7.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/995+Page+8.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/995+Page+9.png'], ['https://cdn.imageart.online/file/CDN-M-A-N/995+Page+10.png'], ['https://cdn.imageart.online/file/CD

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/One_Piece_-_Chapter_996_-_


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

/content/Boku_no_Hero_Academia_-_Chapter_280_-_


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded chapters: 
['One_Piece_-_Chapter_995_-_', 'One_Piece_-_Chapter_996_-_', 'Boku_no_Hero_Academia_-_Chapter_260_-_', 'Boku_no_Hero_Academia_-_Chapter_261_-_', 'Boku_no_Hero_Academia_-_Chapter_262_-_', 'Boku_no_Hero_Academia_-_Chapter_263_-_', 'Boku_no_Hero_Academia_-_Chapter_264_-_', 'Boku_no_Hero_Academia_-_Chapter_265_-_', 'Boku_no_Hero_Academia_-_Chapter_266_-_', 'Boku_no_Hero_Academia_-_Chapter_267_-_', 'Boku_no_Hero_Academia_-_Chapter_268_-_', 'Boku_no_Hero_Academia_-_Chapter_269_-_', 'Boku_no_Hero_Academia_-_Chapter_270_-_', 'Boku_no_Hero_Academia_-_Chapter_271_-_', 'Boku_no_Hero_Academia_-_Chapter_272_-_', 'Boku_no_Hero_Academia_-_Chapter_273_-_', 'Boku_no_Hero_Academia_-_Chapter_274_-_', 'Boku_no_Hero_Academia_-_Chapter_275_-_', 'Boku_no_Hero_Academia_-_Chapter_276_-_', 'Boku_no_Hero_Academia_-_Chapter_277_-_', 'Boku_no_Hero_Academia_-_Chapter_278_-_', 'Boku_no_Hero_Academia_-_Chapter_279_-_', 'Boku_no_Hero_Academia_-_Chapter_280_-_']


In [68]:
# Delete everything in the Google Drive Folder

# Shell command always run first before the python in-line
for i in range(len(completed)):
    os.system('rm -rf ' + completed[i])
    os.system('rm ' + completed[i] + '.zip')
os.chdir(hpath)
completed = []
chapters = []
!ls

sample_data
